## 实现LSTM

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
tf.__version__

'2.3.1'

In [3]:
# !pip install zh-dataset-inews

In [4]:
from zh_dataset_inews import title_train, label_train, content_train, title_test, label_test, content_test

In [5]:
len(title_train)

5355

In [6]:
title_train[:10]

['周六晚到卖场听夜场摇滚',
 '北京老教授泄露，持有山河药辅节后下跌公告，速速看看！！！',
 '张滩镇积极开展基干民兵训练活动',
 '俩小伙无证骑摩托，未成年还试图闯卡！',
 '不好意思，你不配做深圳人!_搜狐汽车_搜狐网',
 '蔡英文元旦升旗遇抗议 民众：枪毙蔡英文',
 '巢湖市绞吸机械清淤公司重在回访-照明器材项目合作–光波网',
 '出租屋半年被偷8次：整栋楼共用一个锁芯',
 '从林芝到拉萨，还可以这样玩!',
 '为何说奇瑞是技术达人? 看了“雄狮”你就懂了']

In [7]:
content_train[:10]

['中央路红星开启“爱木”狂购  三伏天已到，比天气更热的是红星美凯龙的第三届鲁班文化节。这个周末，想避暑的，想找童趣的，想听摇滚的，想买家具的，都可以去红星美凯龙中央路商场。  7月18日18:00-21:00，中央路红星爱木联盟的“仲夏夜party”将闪亮登场。既然是派对，当然少不了音乐，届时将有南京知名摇滚乐队现场驻唱，high唱“红苹果”国际潮流音乐。想抢购家具的消费者，可以通过本报专线025-*******提前预约夜购狂欢。凡成功预约报名的，可在活动签到现场获得夜购预约卡集，凭卡集可领取莫斯利安酸奶一箱或价值150元的丁渤验房券1张，礼品数量有限，领完即止。同时，业主凭卡集还可到夜场活动展厅，尊享百款夜购爆款商品。  作为本次国际潮流音乐之夜的全程赞助商，知名板式品牌红苹果联合红星推出“双红惠”活动，回报顾客。7月6日-19日，顾客关注“红苹果家具”微信公众号可抢红包，另外还可享全屋定制家具优惠，更多活动详情可见中央路红星店堂公告。  暑期活动怎能少得了孩子？近日，儿童家具品牌酷漫居进入中央路红星，7月18日-19日10:20-17:00，儿童t台秀、芭蕾舞表演、小丑游戏、儿童分房讲座等活动精彩纷呈。活动期间还另设儿童活动区域，钓鱼、橡皮泥手工、决明子沙池等着小朋友来玩耍。参与活动的小朋友签到还有精美学习套装赠送，数量有限，送完为止。  这个双休日，到中央路红星买家具的大朋友也有惊喜哦！成交价基础上最高直降5%。此外，会员购物积分将前所未有地直接翻10倍！凭会员卡及当日满额订单扣减积分还可兑换刮刮卡，冷藏箱、塔式电风扇、精美脸谱u盘、莫斯利安酸奶、五常大米、咖啡饮品券、非转基因食用油、书籍……100%有奖，3000份好礼等你抢。陈燕飞\n',
 '大家好，我是老王，一个地地道道的老股民 ，有过巅峰，也有过低谷，最高时期单月盈利近三百万 ，最低时期亏损超过百分之八十 ！十几年的炒股生涯下来，现在的账户也有当时的数十万，变成了现在千万，因为直播抓涨 股的原因很多人都认识了我，直播挂单也与众多股友结识，想必了解我的朋友都是知道的，我只对强势股票感兴趣，个股没有30%以上的操 作空间，我都是不屑一顾，近期公布的个股涨幅都超过了30%。声明一点： 老王非神，不奢求百战百胜，股票终究是七亏一平两盈利的游戏，能帮助70%的股友们盈 利是老王奋斗的目标！最近经常跟我交流的

In [8]:
label_train[:10]

[1, 1, 0, 2, 2, 2, 1, 2, 1, 1]

In [9]:
for x, y in zip(title_train[:10], label_train[:10]):
    print(x, y)

周六晚到卖场听夜场摇滚 1
北京老教授泄露，持有山河药辅节后下跌公告，速速看看！！！ 1
张滩镇积极开展基干民兵训练活动 0
俩小伙无证骑摩托，未成年还试图闯卡！ 2
不好意思，你不配做深圳人!_搜狐汽车_搜狐网 2
蔡英文元旦升旗遇抗议 民众：枪毙蔡英文 2
巢湖市绞吸机械清淤公司重在回访-照明器材项目合作–光波网 1
出租屋半年被偷8次：整栋楼共用一个锁芯 2
从林芝到拉萨，还可以这样玩! 1
为何说奇瑞是技术达人? 看了“雄狮”你就懂了 1


In [10]:
text_vector = tf.keras.layers.experimental.preprocessing.TextVectorization()
# 学习词表
# text_vector.adapt(content_train)
text_vector.adapt(title_train)


In [11]:
# inputs = text_vector(content_train)
inputs = text_vector(title_train)

In [12]:
vocabs = text_vector.get_vocabulary()

vocab_size = len(vocabs)
vocab_size

6909

In [13]:
input_label =  tf.convert_to_tensor(label_train)

In [14]:
embedding_dim = 1024

# model = tf.keras.Sequential([
#     # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
#     tf.keras.layers.Embedding(vocab_size, embedding_dim),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
# #    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#     # use ReLU in place of tanh function since they are very good alternatives of each other.
#     tf.keras.layers.Dense(embedding_dim, activation='relu'),
#     # Add a Dense layer with 6 units and softmax activation.
#     # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
#     tf.keras.layers.Dense(6, activation='softmax')
# ])


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 1024)        7074816   
_________________________________________________________________
bidirectional (Bidirectional (None, 2048)              16785408  
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 24,122,625
Trainable params: 24,122,625
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [ ]:
num_epochs = 10
history = model.fit(
    x = inputs, 
    y = input_label, 
    epochs = num_epochs, 
    validation_split = 0.2,
#     verbose = 2
)

Epoch 1/10


In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()


plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
input_title_test = text_vector(title_test)
input_label_test = tf.convert_to_tensor(label_test)

In [ ]:
test_loss, test_acc = model.evaluate(input_title_test, input_label_test)

In [ ]:
print(test_loss, test_acc)